In [12]:
import pandas as pd
import os
import re
from typing import List
import numpy as np
import matplotlib.pyplot as plt

from bs4 import BeautifulSoup
import requests
import datetime
import time

# 기상청 API 설명
https://apihub.kma.go.kr/



# AWS2 매분자료
https://apihub.kma.go.kr/api/typ01/cgi-bin/url/nph-aws2_min?

https://apihub.kma.go.kr/api/typ01/cgi-bin/url/nph-aws2_min?tm2=202302010900&stn=0&disp=0&help=1&authKey=LcdJI2t7SuCHSSNrexrgGA

MD_column = [ 'YYMMDDHHMI_KST', 'STN_ID', 'WD1_deg', 'WS1_m/s', 'WDS_deg', 'WSS_m/s', 'WS10_deg', 'WS10_m/s', 'TA_C', 
'RE_1', 'RN-15m_mm', 'RN-60m_mm', 'RN-12H_mm', 'RN-DAY_mm', 'HM_%', 'PA_hPa', 'PS_hPa', 'TD_C' ]


# AWS2 매분 시정 자료(현천도 나옴)
https://apihub.kma.go.kr/api/typ01/cgi-bin/url/nph-aws2_min_vis?

https://apihub.kma.go.kr/api/typ01/cgi-bin/url/nph-aws2_min_vis?tm2=202304141510&stn=548&disp=0&help=1&authKey=LcdJI2t7SuCHSSNrexrgGA

MD_NW_column = [ 'YYMMDDHHMI_KST',  'STN_ID', 'LON_deg', 'LAT_deg  S', 'VIS1_m', 'VIS10_m', 'WW1', 'WW15' ]


# AWS2 매분 현천 자료(추가 처리 필요)
https://apihub.kma.go.kr/api/typ01/cgi-bin/url/nph-aws2_min_ww1?

https://apihub.kma.go.kr/api/typ01/cgi-bin/url/nph-aws2_min_ww1?tm1=202304141200&tm2=202304141240&itv=10&range=240&stn=548&help=1&authKey=LcdJI2t7SuCHSSNrexrgGA

MD_NW_column_s = ['YYMMDDHHMI_KST',  'S'TN_ID', 'LON_deg', ' LAT_deg', 'S', 'N', 
'WW1', 'NN1', 'WW2', 'NN2', 'WW3', 'NN3', 'WW4', 'NN4', 'WW5', 'NN5', 'WW6', 'NN6', 'WW7', 'NN7', 'WW8', 'NN8']


In [2]:
def now_data_load(stn, api_key, process = True):
    #현재 기상 데이터를 리턴합니다.
    '''
    MD_column
    
    'YYMMDDHHMI_KST', 'STN_ID', 'WD1_deg', 'WS1_m/s', 'WDS_deg', 'WSS_m/s', 'WS10_deg', 'WS10_m/s', 'TA_C', 
    'RE_1', 'RN-15m_mm', 'RN-60m_mm', 'RN-12H_mm', 'RN-DAY_mm', 'HM_%', 'PA_hPa', 'PS_hPa', 'TD_C'
    '''
    u = 'https://apihub.kma.go.kr/api/typ01/cgi-bin/url/nph-aws2_min?'
    stn = 'stn=' + stn
    api_key = 'authKey=' + api_key

    url = u + '&' + stn  + '&' + api_key
    print(url)
    res = requests.get(url)
    #res.raise_for_status() #정보 출력
    #time.sleep(1)

    soup = BeautifulSoup(res.text, 'html')
    front = soup.text.split('\n')

    real  = []
    for i in range(len(front)-1): #마지막에 줄바꿈 문자가 있어서 인덱싱 에러가 나므로 마지막줄을 빼준다.
        if(front[i][0] != "#"):
            real.append(front[i])
    
    if process == True:
        processed_list = str(real).split()
        
        #첫번째 원소, 마지막 원소에  대괄호가 붙으므로 잘라준다.
        processed_list[0] = processed_list[0][2:]
        processed_list[-1] = processed_list[-1][:-2]

        return processed_list

    return real

In [3]:
# aaZrhx_QRmyma4cf0PZsug
p = now_data_load('548','aaZrhx_QRmyma4cf0PZsug',False)
print (p )
print (type(p ))
p[0]

https://apihub.kma.go.kr/api/typ01/cgi-bin/url/nph-aws2_min?&stn=548&authKey=aaZrhx_QRmyma4cf0PZsug
['202307011621   548  291.9    0.8  284.1    1.1  270.6    1.1   31.7    0.0    0.0    0.0    0.0    0.0   58.3  -99.7  -99.7   22.5']
<class 'list'>


'202307011621   548  291.9    0.8  284.1    1.1  270.6    1.1   31.7    0.0    0.0    0.0    0.0    0.0   58.3  -99.7  -99.7   22.5'

In [4]:
p = now_data_load('548','aaZrhx_QRmyma4cf0PZsug')
print (p, type(p))

https://apihub.kma.go.kr/api/typ01/cgi-bin/url/nph-aws2_min?&stn=548&authKey=aaZrhx_QRmyma4cf0PZsug
['202307011621', '548', '291.9', '0.8', '284.1', '1.1', '270.6', '1.1', '31.7', '0.0', '0.0', '0.0', '0.0', '0.0', '58.3', '-99.7', '-99.7', '22.5'] <class 'list'>


In [5]:
def now_weather_load(stn, api_key, process = True):
    #현재 시정과 현천 데이터를 리턴합니다.
    '''
    MD_NW_column = 'YYMMDDHHMI_KST',  'STN_ID', 'LON_deg', 'LAT_deg  S', 'VIS1_m', 'VIS10_m', 'WW1', 'WW15'
    '''
    u = 'https://apihub.kma.go.kr/api/typ01/cgi-bin/url/nph-aws2_min_vis?'
    stn = 'stn=' + stn
    api_key = 'authKey=' + api_key

    url = u + '&' + stn  + '&' + api_key
    print(url)
    res = requests.get(url)
    #res.raise_for_status() #정보 출력
    #time.sleep(1)

    soup = BeautifulSoup(res.text, 'html')
    front = soup.text.split('\n')

    real  = []
    for i in range(len(front)-1): #마지막에 줄바꿈 문자가 있어서 인덱싱 에러가 나므로 마지막줄을 빼준다.
        if(front[i][0] != "#"):
            real.append(front[i])
            
    if process == True:
        processed_list = str(real).split()
        
        #첫번째 원소, 마지막 원소에  대괄호가 붙으므로 잘라준다.
        processed_list[0] = processed_list[0][2:]
        processed_list[-1] = processed_list[-1][:-2]

        return processed_list

    return real

In [6]:
p = now_weather_load('548','aaZrhx_QRmyma4cf0PZsug',False)

print (p, type(p))
print(p[0], type(p[0]))
p[0]

https://apihub.kma.go.kr/api/typ01/cgi-bin/url/nph-aws2_min_vis?&stn=548&authKey=aaZrhx_QRmyma4cf0PZsug
['202307011621   548 127.63955688  37.26882935  2  31650   -999      0   -999'] <class 'list'>
202307011621   548 127.63955688  37.26882935  2  31650   -999      0   -999 <class 'str'>


'202307011621   548 127.63955688  37.26882935  2  31650   -999      0   -999'

In [7]:
p = now_weather_load('548','aaZrhx_QRmyma4cf0PZsug')
print (p, type(p))

https://apihub.kma.go.kr/api/typ01/cgi-bin/url/nph-aws2_min_vis?&stn=548&authKey=aaZrhx_QRmyma4cf0PZsug
['202307011621', '548', '127.63955688', '37.26882935', '2', '31650', '-999', '0', '-999'] <class 'list'>


In [8]:
import datetime

def find_after_min(now , delta_min):
    #now 에서 delta_min 만큼 변화한 시간을 문자열로 리턴한다.

    #### 입력 #### 
    # now = '202304151420' 와 같은 형식의 문자열
    # delta_min = 시간 변화(분 단위의 정수)

    now = datetime.datetime.strptime(now,'%Y%m%d%H%M')
    past = (now + datetime.timedelta(minutes=delta_min)).strftime('%Y%m%d%H%M')
    
    return past

In [9]:
ㅅ = find_after_min('202304151410' ,-240) 
print ( ㅅ ) 
print ( type( ㅅ ))


202304151010
<class 'str'>


In [10]:
def past_weather_load(stn, api_key, date):
    # date 기준으로 과거 4시간 동안의 기록을 분 단위로 얻어오기.(480 행)
    # 튜플 리턴
    # 첫번째 원소 => 넘파이 배열(현천의 고유값들)
    # 두번째 원소 => 넘파이 배열(해당 고유값 별 갯수)
    
    now = datetime.datetime.strptime(date,'%Y%m%d%H%M')
    past = (now + datetime.timedelta(hours=-4)).strftime('%Y%m%d%H%M')
    print(now.strftime('%Y%m%d%H%M'), '~', past)

    url = f'https://apihub.kma.go.kr/api/typ01/cgi-bin/url/nph-aws2_min_vis?tm1={past}&tm2={date}&stn={stn}&disp=0&help=1&authKey={api_key}'
    print(url)
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'html')
    front = soup.text.split('\n')

    real  = []
    for i in range(len(front)-1): #마지막에 줄바꿈 문자가 있어서 인덱싱 에러가 나므로 마지막줄을 빼준다.
        if(front[i][0] != "#"):
            front[i] = front[i].split()
            real.append(front[i][-2])
    #print(real)
    #print(len(real))

    return np.unique(real, return_counts=True)


In [11]:
now = datetime.datetime.now().strftime('%Y%m%d%H%M')
print(now)
dff = past_weather_load('548', 'aaZrhx_QRmyma4cf0PZsug', now)
print(dff, type(dff))

202307011624
202307011624 ~ 202307011224
https://apihub.kma.go.kr/api/typ01/cgi-bin/url/nph-aws2_min_vis?tm1=202307011224&tm2=202307011624&stn=548&disp=0&help=1&authKey=aaZrhx_QRmyma4cf0PZsug


(array(['0'], dtype='<U1'), array([239], dtype=int64)) <class 'tuple'>


In [12]:
now = datetime.datetime.strptime('202304151250','%Y%m%d%H%M').strftime('%Y%m%d%H%M')
print(now)

dff = past_weather_load('548', 'aaZrhx_QRmyma4cf0PZsug', now)
print(dff, type(dff))

202304151250
202304151250 ~ 202304150850
https://apihub.kma.go.kr/api/typ01/cgi-bin/url/nph-aws2_min_vis?tm1=202304150850&tm2=202304151250&stn=548&disp=0&help=1&authKey=aaZrhx_QRmyma4cf0PZsug
(array(['0', '51', '61', '62', '89'], dtype='<U2'), array([174,   3,  14,  48,   2], dtype=int64)) <class 'tuple'>


In [13]:
def weather_parse(info):
    '''
    info = 다음과 같은 입력 자료
    info= ('202304151250', '548', '127.63955688', '37.26882935', '2', '2', '61', '2', '62', '3', '=') # tuple
    info = ['202304151250', '548', '127.63955688', '37.26882935', '2', '2', '61', '2', '62', '3', '='] # list
    info = ['202304151150', '548', '127.63955688', '37.26882935', '2', '2', '51', '1', '61', '4', '='] 
    info = ['202304151020', '548', '127.63955688', '37.26882935', '2', '1', '0', '5', '=']

    날씨 카테고리별 관측횟수 리턴.
    
    'YYMMDDHHMI_KST',  'STN_ID', 'LON_deg', ' LAT_deg', 'S', 'N', 
    'WW1', 'NN1', 'WW2', 'NN2', 'WW3', 'NN3', 'WW4', 'NN4', 'WW5', 'NN5', 'WW6', 'NN6', 'WW7', 'NN7', 'WW8', 'NN8'

    WW 는 날씨 코드, NN은 해당 코드가 관측된 횟수
    코드 : 0~2(맑음), 4(연무), 10(박무), 30(안개), 40~42(비), 50~59(안개비), 60~68(비), 71~76(눈)
    '''

    WW_NN = np.zeros(8) # 날시 상태별로 관측된 횟수를 기록할 변수. (8종류)
    state_infomation = info[5:-1]
    repeat = int(state_infomation[0]) #관측된 날씨의 종류 수

    i = 1
    k =1
    while k:
        #2 개씩 묶어서 분석.
        state = int(state_infomation[i])
        number = int(state_infomation[i+1])

        if 0 <= state <= 2:
            WW_NN[0] = WW_NN[0] + number

        elif state == 4:
            WW_NN[1] = WW_NN[1] + number

        elif state == 10:
            WW_NN[2] = WW_NN[2] + number

        elif state == 30:
            WW_NN[3] = WW_NN[3] + number

        elif 40 <= state <= 42:
            WW_NN[4] = WW_NN[4] + number

        elif 50 <= state <= 59:
            WW_NN[5] = WW_NN[5] + number

        elif 60 <= state <= 68:
            WW_NN[6] = WW_NN[6] + number

        elif 71 <= state <= 76:
            WW_NN[7] = WW_NN[7] + number

        i = i+2
        k = k +1
        if(k >= repeat+1):
            break

    
    return WW_NN

In [14]:
x = ('202304151250', '548', '127.63955688', '37.26882935', '2', '2', '61', '2', '62', '3', '=') # tuple
y = ['202304151250', '548', '127.63955688', '37.26882935', '2', '2', '61', '2', '62', '3', '='] # list
z = ['202304151150', '548', '127.63955688', '37.26882935', '2', '2', '51', '1', '61', '4', '='] 
u = ['202304151020', '548', '127.63955688', '37.26882935', '2', '1', '0', '5', '=']
print(weather_parse(x))
print(weather_parse(y))
print(weather_parse(z))
print(weather_parse(u))

[0. 0. 0. 0. 0. 0. 5. 0.]
[0. 0. 0. 0. 0. 0. 5. 0.]
[0. 0. 0. 0. 0. 1. 4. 0.]
[5. 0. 0. 0. 0. 0. 0. 0.]


In [15]:
def past_aws2_weather_load(stn, api_key, end, p=False):
    #과거 4시간 이내의 현천 데이터를 리턴합니다. (5분 간격)
    '''
    MD_NW_column_s 
    
    'YYMMDDHHMI_KST',  'STN_ID', 'LON_deg', ' LAT_deg', 'S', 'N', 
    'WW1', 'NN1', 'WW2', 'NN2', 'WW3', 'NN3', 'WW4', 'NN4', 'WW5', 'NN5', 'WW6', 'NN6', 'WW7', 'NN7', 'WW8', 'NN8'
    '''
    u = 'https://apihub.kma.go.kr/api/typ01/cgi-bin/url/nph-aws2_min_ww1?'
    stn = 'stn=' + stn
    api_key = 'authKey=' + api_key

    start = find_after_min(end , -240) #과거 4시간이므로 -240분을 시작 구간으로 지정
    start = 'tm1=' + start
    end = 'tm2=' + end #현재시간을 끝 구간으로 지정

    url = u + start +'&' + end + '&' + 'itv=5&range=5&'+ stn  + '&' + api_key
    print(url)
    res = requests.get(url)
    #res.raise_for_status() #정보 출력
    #time.sleep(1)

    soup = BeautifulSoup(res.text, 'html')
    front = soup.text.split('\n')

    real  = []
    for i in range(len(front)-1): #마지막에 줄바꿈 문자가 있어서 인덱싱 에러가 나므로 마지막줄을 빼준다.
        if(front[i][0] != "#"):
            real.append(front[i])

    for i in range(len(real)):
        real[i] = str(real[i]).split(',') # 슬라이싱
        for k in range(len(real[i])): # 공백제거. (기상청 API 출력에 공백이 제멋대로 삽입되어 있음
            real[i][k] = real[i][k].replace(" ", "")
    
    WW_NN = np.zeros(8)

    for i in range(len(real)): # 예쁘게 날씨 종류별 관측 횟수 통계 내기
        if(p): print(real[i])
        WW_NN = WW_NN + weather_parse(real[i])
    print('\n')
    
    return WW_NN

In [16]:
q = past_aws2_weather_load('548', 'aaZrhx_QRmyma4cf0PZsug', '202304151420')
print ( q )
print ( type( q ) )

https://apihub.kma.go.kr/api/typ01/cgi-bin/url/nph-aws2_min_ww1?tm1=202304151020&tm2=202304151420&itv=5&range=5&stn=548&authKey=aaZrhx_QRmyma4cf0PZsug


[ 88.   0.   0.   0.   0.   3. 152.   0.]
<class 'numpy.ndarray'>


In [17]:
STN = '548'
API_KEY = 'aaZrhx_QRmyma4cf0PZsug'
time = datetime.datetime.now().strftime('%Y%m%d%H%M')
q = past_aws2_weather_load(STN, API_KEY, time, True)
print ( q )
print ( type( q ))

https://apihub.kma.go.kr/api/typ01/cgi-bin/url/nph-aws2_min_ww1?tm1=202307011224&tm2=202307011624&itv=5&range=5&stn=548&authKey=aaZrhx_QRmyma4cf0PZsug
['202307011220', '548', '127.63955688', '37.26882935', '2', '1', '0', '5', '=']
['202307011225', '548', '127.63955688', '37.26882935', '2', '1', '0', '5', '=']
['202307011230', '548', '127.63955688', '37.26882935', '2', '1', '0', '5', '=']
['202307011235', '548', '127.63955688', '37.26882935', '2', '1', '0', '5', '=']
['202307011240', '548', '127.63955688', '37.26882935', '2', '1', '0', '5', '=']
['202307011245', '548', '127.63955688', '37.26882935', '2', '1', '0', '5', '=']
['202307011250', '548', '127.63955688', '37.26882935', '2', '1', '0', '5', '=']
['202307011255', '548', '127.63955688', '37.26882935', '2', '1', '0', '5', '=']
['202307011300', '548', '127.63955688', '37.26882935', '2', '1', '0', '5', '=']
['202307011305', '548', '127.63955688', '37.26882935', '2', '1', '0', '5', '=']
['202307011400', '548', '127.63955688', '37.26882

In [18]:
def predict_part_load(local, api_key):
    #예보 구역 이름과 api 키를 입력받아서
    #현재 지역의 예보 구역 코드를 리턴. 
    '''
    REG_ID   : 예보구역코드
    TM_ST    : 시작시각(년월일시분,KST)
    TM_ED    : 종료시각(년월일시분,KST)
    REG_SP   : 특성 (A:육상광역,B:육상국지,C:도시,D:산악,E:고속도로,H:해상광역,I:해상국지,J:연안바다,K:해수욕장,L:연안항로,M:먼항로,P:산악)
    REG_NAME : 예보구역명


    11B20701 201610131800 210012310000 C      이천 
    11B20702 201610131800 210012310000 C      광주 
    11B20703 201610131800 210012310000 C      여주 
    11C000D1 199001010000 210012310000 D      계룡산
    '''

    url = f'https://apihub.kma.go.kr/api/typ01/url/fct_shrt_reg.php?tmfc=0&authKey={api_key}'
    
    res = requests.get(url)
    print(url)
    #time.sleep(1)

    soup = BeautifulSoup(res.text, 'html')
    front = soup.text.split('\n')

    real  = []
    for i in range(len(front)-1): #마지막에 줄바꿈 문자가 있어서 인덱싱 에러가 나므로 마지막줄을 빼준다.
        if(front[i][0] != "#"):
            real.append(front[i])

    location = None

    for i in range(len(real)):
        real[i] = str(real[i]).split(' ') # 슬라이싱
        for k in range(len(real[i])):
            if real[i][k] == local:
                location = real[i]

    if location == None: 
        print('해당 지역 정보가 없음.')
        return None

    info = []
    for i in range(len(location)):
        if len(location[i]) > 0:
            info.append(location[i])

    return info

In [19]:
p = predict_part_load('여주', 'aaZrhx_QRmyma4cf0PZsug')
print(p)
print(type(p))

https://apihub.kma.go.kr/api/typ01/url/fct_shrt_reg.php?tmfc=0&authKey=aaZrhx_QRmyma4cf0PZsug
['11B20703', '201610131800', '210012310000', 'C', '여주']
<class 'list'>


In [20]:
p = predict_part_load('ㅇㅇ', 'aaZrhx_QRmyma4cf0PZsug')
print(p)
print(type(p))

https://apihub.kma.go.kr/api/typ01/url/fct_shrt_reg.php?tmfc=0&authKey=aaZrhx_QRmyma4cf0PZsug
해당 지역 정보가 없음.
None
<class 'NoneType'>


In [21]:
#https://apihub.kma.go.kr/api/typ01/url/fct_afs_dl2.php?reg=11B20703&tmfc1=202304200000&tmfc2=202304210000&&disp=0&help=1&authKey=Os5DnJkMQpmOQ5yZDLKZ2Q

##기상청 날씨누리 단기 에보.

https://www.weather.go.kr/w/weather/forecast/short-term.do#




##기상청 초단기예보 API(기상청 API 허브에 나온 API랑 다름.)

https://www.data.go.kr/iim/api/selectAPIAcountView.do

In [1]:
def predict_time_fix(time_str):
    # 기상청 예보 생성 시간별로 제공해주는 예보 시각이 다르다.
    # 단기예보
    # Base_time : 0200, 0500, 0800, 1100, 1400, 1700, 2000, 2300 (1일 8회)
    # API 제공 시간(~이후) : 02:10, 05:10, 08:10, 11:10, 14:10, 17:10, 20:10, 23:10
    time_str = str(time_str)
    day = time_str[:8]
    time = time_str[8:]

    if int(time) < 210:
        day = ( datetime.datetime.now() - datetime.timedelta(1) ).strftime('%Y%m%d')
        time = '2300'

    if 210 <= int(time) <= 510:
        time = '0200'
    if 511 <= int(time) <= 810:
        time = '0500'
    if 811 <= int(time) <= 1110:
        time = '0800'
    if 1111 <= int(time) <= 1410:
        time = '1100'
    if 1411 <= int(time) <= 1710:
        time = '1400'
    if 1711 <= int(time) <= 2010:
        time = '1700'
    if 2011 <= int(time) <= 2310:
        time = '2000'
    if 2311 <= int(time) <= 2359:
        time = '2300'

    #fix_time_str = day + time

    return day,  time

In [23]:
ts = predict_time_fix('202304251410')
print( ts )

('20230425', '1100')


In [3]:
def predict_data_load(api_key2, today, today_time, lx, ly):
    #http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getVilageFcst
    '''
    api_key2, today, today_time, lx, ly

    api_key2 = 'eqTbu5tgsBlxHlliXVpp4A6FZyGJCS6ct%2FQWVPZuhRy3x05IGhAHlr0U9b%2B4qPEUNfrWVf2n5FtdffkCi33MQg%3D%3D' #설정파일로 설정할 값
    today = '20230421' # 현재 날짜로 설정할 값, 2~3일 전 날짜 사용시 api 리턴 결과 없음 주의
    today_time = '0537' # 현재 시간으로 설정할 값
    location_x = '55'  # 격자구역 x 정보
    location_y = '127' # 격자구역 y 정보

    api 리턴값 예시
    <items>
        <item>
            <baseDate>20230422</baseDate> #공통
            <baseTime>0500</baseTime> #공통
            <category>TMP</category> #카테고리, 유일값
            <fcstDate>20230422</fcstDate> #공통
            <fcstTime>0600</fcstTime> #공통
            <fcstValue>9</fcstValue> #실제 값
            <nx>55</nx> #공통
            <ny>127</ny> #공통
        </item>
    <item> 

    # 이상이 시간, 카테고리별로 반복된다. 매 시간(정시) 별 자료를 받을 수 있으며, 각 시간당 12개의 항목이 있다. 
    예시. # data = soup.select('item')
    202304220500 TMP 202304220600 9 55 127 #1시간 기온/ 10 / ºC
    202304220500 UUU 202304220600 -1.3 55 127 #풍속(동서성분)/ 12 / m/s
    202304220500 VVV 202304220600 -0.9 55 127 #풍속(남북성분)/ 12 / m/s
    202304220500 VEC 202304220600 57 55 127 #풍향/ 10/ deg
    202304220500 WSD 202304220600 1.7 55 127 #풍속/ 10/ m/s
    202304220500 SKY 202304220600 4 55 127 #하늘 상태/ 4/ 코드값
    202304220500 PTY 202304220600 0 55 127 #강수형태/ 4 / 코드값
    202304220500 POP 202304220600 30 55 127 #강수확률/ 8 / %
    202304220500 WAV 202304220600 0 55 127 #파고/ 8/ M
    202304220500 PCP 202304220600 강수없음 55 127 #1시간 강수량/ 8/ 범주(1 mm)
    202304220500 REH 202304220600 60 55 127 #습도/ 8 / %
    202304220500 SNO 202304220600 적설없음 55 127 #1시간 신적설/ 8/ 범주(1 cm)

    '''
    today, today_time = predict_time_fix(today + today_time)
    url = f'http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getVilageFcst?serviceKey={api_key2}&numOfRows=48&pageNo=1&base_date={today}&base_time={today_time}&nx={lx}&ny={ly}'  
    print(url)
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'xml')
    category = soup.select('category')
    fcstValue = soup.select('fcstValue')
    
    #print(soup)
    data = []
    for it in range(len(category)):
        #print( category[it].text)
        data.append(fcstValue[it].text)

    data = np.array(data).reshape(-1, 12)

    return data

In [5]:
def predict_data_load_L(list):
    #http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getVilageFcst

    '''
    api_key2, today, today_time, lx, ly

    api_key2 = 'eqTbu5tgsBlxHlliXVpp4A6FZyGJCS6ct%2FQWVPZuhRy3x05IGhAHlr0U9b%2B4qPEUNfrWVf2n5FtdffkCi33MQg%3D%3D' #설정파일로 설정할 값
    today = '20230421' # 현재 날짜로 설정할 값, 2~3일 전 날짜 사용시 api 리턴 결과 없음 주의
    today_time = '0537' # 현재 시간으로 설정할 값
    location_x = '55'  # 격자구역 x 정보
    location_y = '127' # 격자구역 y 정보

    
    api 리턴값 예시
    <items>
        <item>
            <baseDate>20230422</baseDate> #공통
            <baseTime>0500</baseTime> #공통
            <category>TMP</category> #카테고리, 유일값
            <fcstDate>20230422</fcstDate> #공통
            <fcstTime>0600</fcstTime> #공통
            <fcstValue>9</fcstValue> #실제 값
            <nx>55</nx> #공통
            <ny>127</ny> #공통
        </item>
    <item> 

    # 이상이 시간, 카테고리별로 반복된다. 매 시간(정시) 별 자료를 받을 수 있으며, 각 시간당 12개의 항목이 있다. 
    예시. # data = soup.select('item')
    202304220500 TMP 202304220600 9 55 127 #1시간 기온/ 10 / ºC
    202304220500 UUU 202304220600 -1.3 55 127 #풍속(동서성분)/ 12 / m/s
    202304220500 VVV 202304220600 -0.9 55 127 #풍속(남북성분)/ 12 / m/s
    202304220500 VEC 202304220600 57 55 127 #풍향/ 10/ deg
    202304220500 WSD 202304220600 1.7 55 127 #풍속/ 10/ m/s
    202304220500 SKY 202304220600 4 55 127 #하늘 상태/ 4/ 코드값
    202304220500 PTY 202304220600 0 55 127 #강수형태/ 4 / 코드값
    202304220500 POP 202304220600 30 55 127 #강수확률/ 8 / %
    202304220500 WAV 202304220600 0 55 127 #파고/ 8/ M
    202304220500 PCP 202304220600 강수없음 55 127 #1시간 강수량/ 8/ 범주(1 mm)
    202304220500 REH 202304220600 60 55 127 #습도/ 8 / %
    202304220500 SNO 202304220600 적설없음 55 127 #1시간 신적설/ 8/ 범주(1 cm)

    '''
    api_key2, today, today_time, lx, ly = list
    today, today_time = predict_time_fix(today + today_time)

    url = f'http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getVilageFcst?serviceKey={api_key2}&numOfRows=60&pageNo=1&base_date={today}&base_time={today_time}&nx={lx}&ny={ly}'  
    print(url)
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'xml')
    category = soup.select('category')
    fcstValue = soup.select('fcstValue')

    data = []
    for it in range(len(category)):
        #print( category[it].text)
        data.append(fcstValue[it].text)

    data = np.array(data).reshape(-1, 12)
    return data

In [6]:
api_key2 = 'eqTbu5tgsBlxHlliXVpp4A6FZyGJCS6ct%2FQWVPZuhRy3x05IGhAHlr0U9b%2B4qPEUNfrWVf2n5FtdffkCi33MQg%3D%3D'
today = '20230828' #현재 날짜로 설정할 값
today_time = '1638' # 현재 시간으로 설정할 값

# 여주 AWS  127.63955688  37.26882935 격자구역 변환 -> 71, 121
location_x = '71'  #격자구역 x
location_y = '121' #격자구역 y

per = [api_key2, today, today_time, location_x, location_y]
k = predict_data_load_L(per)
print(k)
print(type(k))

http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getVilageFcst?serviceKey=eqTbu5tgsBlxHlliXVpp4A6FZyGJCS6ct%2FQWVPZuhRy3x05IGhAHlr0U9b%2B4qPEUNfrWVf2n5FtdffkCi33MQg%3D%3D&numOfRows=60&pageNo=1&base_date=20230828&base_time=1400&nx=71&ny=121
60 60 60 60 60 60
[['20230828 1400 TMP 20230828 1500 26'
  '20230828 1400 UUU 20230828 1500 -3'
  '20230828 1400 VVV 20230828 1500 0.6'
  '20230828 1400 VEC 20230828 1500 101'
  '20230828 1400 WSD 20230828 1500 3.2'
  '20230828 1400 SKY 20230828 1500 4' '20230828 1400 PTY 20230828 1500 1'
  '20230828 1400 POP 20230828 1500 60'
  '20230828 1400 WAV 20230828 1500 0'
  '20230828 1400 PCP 20230828 1500 1.0mm'
  '20230828 1400 REH 20230828 1500 80'
  '20230828 1400 SNO 20230828 1500 적설없음']
 ['20230828 1400 TMP 20230828 1600 26'
  '20230828 1400 UUU 20230828 1600 -2.6'
  '20230828 1400 VVV 20230828 1600 0'
  '20230828 1400 VEC 20230828 1600 90'
  '20230828 1400 WSD 20230828 1600 2.7'
  '20230828 1400 SKY 20230828 1600 4' '20230828 1400 PTY 20230828

In [27]:
k = predict_data_load(api_key2, today, today_time, location_x, location_y)
print(k)
print(type(k))

http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getVilageFcst?serviceKey=eqTbu5tgsBlxHlliXVpp4A6FZyGJCS6ct%2FQWVPZuhRy3x05IGhAHlr0U9b%2B4qPEUNfrWVf2n5FtdffkCi33MQg%3D%3D&numOfRows=48&pageNo=1&base_date=20230630&base_time=2000&nx=71&ny=121
[['25' '-1.1' '0.5' '113' '1.3' '3' '0' '20' '0' '강수없음' '90' '적설없음']
 ['24' '-0.9' '0.4' '112' '1.1' '4' '0' '30' '0' '강수없음' '90' '적설없음']
 ['24' '-1' '0.1' '95' '1.1' '4' '0' '30' '0' '강수없음' '95' '적설없음']
 ['23' '-1' '0.3' '105' '1.1' '3' '0' '20' '0' '강수없음' '95' '적설없음']]
<class 'numpy.ndarray'>


In [2]:
def predict_data_load_L(list):
    api_key2, today, today_time, lx, ly = list
    today, today_time = predict_time_fix(today + today_time)

    url = f'http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getVilageFcst?serviceKey={api_key2}&numOfRows=60&pageNo=1&base_date={today}&base_time={today_time}&nx={lx}&ny={ly}'  
    print(url)
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'xml')

    #print(soup)
    basedate = soup.select('baseDate')
    basetime = soup.select('baseTime')
    category = soup.select('category')
    fcstdate = soup.select('fcstDate')
    fcsttime = soup.select('fcstTime')
    fcstValue = soup.select('fcstValue')
    #item = soup.select('item')
    print(len(basedate), len(basetime), len(category), len(fcstdate), len(fcsttime), len(fcstValue) )

    data = []
    for it in range(len(basedate)):
        #print( category[it].text)
        data.append(f'{basedate[it].text} {basetime[it].text} {category[it].text} {fcstdate[it].text} {fcsttime[it].text} {fcstValue[it].text}')

    data = np.array(data).reshape(-1, 12)
    return data

In [9]:
per = [api_key2, today, today_time, location_x, location_y]
k = predict_data_load_L(per)
print(k)
print(type(k))
print(len(k))

http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getVilageFcst?serviceKey=eqTbu5tgsBlxHlliXVpp4A6FZyGJCS6ct%2FQWVPZuhRy3x05IGhAHlr0U9b%2B4qPEUNfrWVf2n5FtdffkCi33MQg%3D%3D&numOfRows=60&pageNo=1&base_date=20230828&base_time=1400&nx=71&ny=121
60 60 60 60 60 60
[['20230828 1400 TMP 20230828 1500 26'
  '20230828 1400 UUU 20230828 1500 -3'
  '20230828 1400 VVV 20230828 1500 0.6'
  '20230828 1400 VEC 20230828 1500 101'
  '20230828 1400 WSD 20230828 1500 3.2'
  '20230828 1400 SKY 20230828 1500 4' '20230828 1400 PTY 20230828 1500 1'
  '20230828 1400 POP 20230828 1500 60'
  '20230828 1400 WAV 20230828 1500 0'
  '20230828 1400 PCP 20230828 1500 1.0mm'
  '20230828 1400 REH 20230828 1500 80'
  '20230828 1400 SNO 20230828 1500 적설없음']
 ['20230828 1400 TMP 20230828 1600 26'
  '20230828 1400 UUU 20230828 1600 -2.6'
  '20230828 1400 VVV 20230828 1600 0'
  '20230828 1400 VEC 20230828 1600 90'
  '20230828 1400 WSD 20230828 1600 2.7'
  '20230828 1400 SKY 20230828 1600 4' '20230828 1400 PTY 20230828

In [80]:
def response_state(code: int) -> list:
    if code == 200:
        msg = '요청이 성공적으로 처리됨'

    if code != 200:
        if code < 200: msg = '1xx (Informational): 요청수신, 처리중'
        if code < 300: msg = '2xx (Successful): 요청수신, 처리됨'
        if code < 400: msg = '3xx (Redirection): 추가 작업이 필요'
        if code < 500: msg = '4xx (Client Error): 잘못된 요청'
        if code < 600: msg = '5xx (Server Error): 서버 오류'

    if code == 201:
        msg = '요청으로 새로운 리소스가 생성됨'
    if code == 204:
        msg = '요청이 처리되었으나 응답 없음'
    if code == 400:
        msg = '잘못된 클라이언트 요청'
    if code == 401:
        msg = '유효하지 않은 인증 정보'
    if code == 403:
        msg = '리소스 접근 권한 없음'
    if code == 404:
        msg = '리소스를 찾을 수 없음'
    if code == 500:
        msg = '요청 수행 중 문제 발생'
    if code == 503:
        msg = '서버 오류'
    return [str(code), msg]


def predict_time_fix(time_str: str) -> tuple:
    # 기상청 예보 생성 시간별로 제공해주는 예보 시각이 다르다.
    # 단기예보
    # Base_time : 0200, 0500, 0800, 1100, 1400, 1700, 2000, 2300 (1일 8회)
    # API 제공 시간(~이후) : 02:10, 05:10, 08:10, 11:10, 14:10, 17:10, 20:10, 23:10
    time_str = str(time_str)
    day = time_str[:8]
    time = time_str[8:]

    if int(time) < 210:
        day = ( datetime.datetime.now() - datetime.timedelta(1) ).strftime('%Y%m%d')
        time = '2300'

    if 210 <= int(time) <= 510:
        time = '0200'
    if 511 <= int(time) <= 810:
        time = '0500'
    if 811 <= int(time) <= 1110:
        time = '0800'
    if 1111 <= int(time) <= 1410:
        time = '1100'
    if 1411 <= int(time) <= 1710:
        time = '1400'
    if 1711 <= int(time) <= 2010:
        time = '1700'
    if 2011 <= int(time) <= 2310:
        time = '2000'
    if 2311 <= int(time) <= 2359:
        time = '2300'

    return day,  time



def fcstValue_fix_PCP(pcp: str) -> str:
    replace = None

    if pcp == '강수없음':
        replace = '0'
    if pcp == '1.0mm 미만':
        replace = '0.5'

    match = re.match(r'(\d+\.\d+)mm', pcp) # '실수mm' 형태의 문자열이 감지되면 실수부 추출
    if match:
        replace = match.group(1)

    if pcp == '30.0~50.0mm':
        replace = '30.0'
    if pcp == '50.0mm 이상':
        replace = '51.0'
    
    return replace



def fcstValue_fix_SNO(sno: str) -> str:
    replace = None

    if sno == '적설없음':
        replace = '0'
    if sno == '1.0cm 미만':
        replace = '0.5'

    match = re.match(r'(\d+\.\d+)cm', sno)
    if match:
        replace = match.group(1)

    if sno == '5.0cm 이상':
        replace = '5.1'
    
    return replace

def predict_data_load_L(datalist: List[str]) -> np.ndarray:
    '''
    기준 시간으로부터(현재시간 x) 4시간치의 일기 예보를 얻어온다 \n
    http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getVilageFcst \n
    api_key2 = 'eqTbu5tgsBlxHlliXVpp4A6FZyGJCS6ct%2FQWVPZuhRy3x05IGhAHlr0U9b%2B4qPEUNfrWVf2n5FtdffkCi33MQg%3D%3D'\n
    today = '20230421' # 현재 날짜로 설정할 값, 2~3일 전 날짜 사용시 api 리턴 결과 없음 주의\n
    today_time = '0537' # 현재 시간으로 설정할 값\n
    location_x = '55'  # 격자구역 x 정보\n
    location_y = '127' # 격자구역 y 정보\n

    
    api 리턴값 예시
    <items>
        <item>
            <baseDate>20230422</baseDate> #공통
            <baseTime>0500</baseTime> #공통
            <category>TMP</category> #카테고리, 유일값
            <fcstDate>20230422</fcstDate> #공통
            <fcstTime>0600</fcstTime> #공통
            <fcstValue>9</fcstValue> #실제 값
            <nx>55</nx> #공통
            <ny>127</ny> #공통
        </item>
    <item> 

    # 이상이 시간, 카테고리별로 반복된다. 매 시간(정시) 별 자료를 받을 수 있으며, 각 시간당 12개의 항목이 있다. 
    예시. # data = soup.select('item')
    202304220500 TMP 202304220600 9 55 127 #1시간 기온/ 10 / ºC
    202304220500 UUU 202304220600 -1.3 55 127 #풍속(동서성분)/ 12 / m/s
    202304220500 VVV 202304220600 -0.9 55 127 #풍속(남북성분)/ 12 / m/s
    202304220500 VEC 202304220600 57 55 127 #풍향/ 10/ deg
    202304220500 WSD 202304220600 1.7 55 127 #풍속/ 10/ m/s
    202304220500 SKY 202304220600 4 55 127 #하늘 상태/ 4/ 코드값
    202304220500 PTY 202304220600 0 55 127 #강수형태/ 4 / 코드값
    202304220500 POP 202304220600 30 55 127 #강수확률/ 8 / %
    202304220500 WAV 202304220600 0 55 127 #파고/ 8/ M
    202304220500 PCP 202304220600 강수없음 55 127 #1시간 강수량/ 8/ 범주(1 mm)
    202304220500 REH 202304220600 60 55 127 #습도/ 8 / %
    202304220500 SNO 202304220600 적설없음 55 127 #1시간 신적설/ 8/ 범주(1 cm)


    결과 값을 넘파이 배열 형태로 정리해서 넘겨준다.
    [[10   0.1     0  269  0.1  1  0  0  0  강수없음  65  적설없음]
      [9  -0.5  -0.2   72  0.6  1  0  0  0  강수없음  70  적설없음]
      [9    -1  -0.1   85  1.1  1  0  0  0  강수없음  70  적설없음]
      [7  -1.5     0   90  1.6  1  0  0  0  강수없음  75  적설없음]]
    
    하늘상태(SKY) 코드 : 맑음(1), 구름많음(3), 흐림(4)
    강수형태(PTY) 코드 : (단기) 없음(0), 비(1), 비/눈(2), 눈(3), 소나기(4) 


    강수량(RN1, PCP) 범주 및 표시방법(값)
    0.1 ~ 1.0mm 미만 -> 1.0mm 미만
    1.0mm 이상 30.0mm 미만 -> 실수값+mm (1.0mm~29.9mm)
    30.0 mm 이상 50.0 mm 미만 -> 30.0~50.0mm 
    50.0 mm 이상 -> 50.0mm 이상

    예) PCP = 6.2 일 경우 강수량은 6.2mm
    PCP = 30 일 경우 강수량은 30.0~50.0mm


    동서바람성분(UUU) : 동(+표기), 서(-표기)
    남북바람성분(VVV) : 북(+표기), 남(-표기)



    '''
    api_key2, today, today_time, lx, ly = datalist
    today, today_time = predict_time_fix(today + today_time)
    #print(today, today_time)

    url = f'http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getVilageFcst?serviceKey={api_key2}&numOfRows=61&pageNo=1&base_date={today}&base_time={today_time}&nx={lx}&ny={ly}'  
    #print(url)
    try:
        res = requests.get(url)
    except:
        return ['Exception ', 'Fail Connection']
    if response_state(res.status_code)[0] != str(200):
        return np.ndarray(response_state(res.status_code))
    soup = BeautifulSoup(res.text, features='xml')
    category = soup.select('category')
    fcstValue = soup.select('fcstValue')

    key = []
    data = []

    #replace  = 'default'
    for it in range(len(category)):
        if category[it].text == 'TMN' or category[it].text == 'TMX':
            pass
        else:
            data.append(fcstValue[it].text)

    pp = ''
    for i in range(len(key)):
        if key[i] == 'SNO':
            pp = pp + f' {key[i]}\n'
        else:
            pp = pp + f' {key[i]},'
    print(pp)
    print(np.array(key).reshape(-1, 12))
    print(np.array(data).reshape(-1, 12))

    #data = np.array(data).reshape(-1, 12)

    #for i in range(len(data)):
        #data[i][9] = fcstValue_fix_PCP(data[i][9])
        #data[i][11] = fcstValue_fix_SNO(data[i][11])
    return data

In [81]:
api_key2 = 'eqTbu5tgsBlxHlliXVpp4A6FZyGJCS6ct%2FQWVPZuhRy3x05IGhAHlr0U9b%2B4qPEUNfrWVf2n5FtdffkCi33MQg%3D%3D'
today = '20230829' #현재 날짜로 설정할 값
today_time = '0247' # 현재 시간으로 설정할 값

# 여주 AWS  127.63955688  37.26882935 격자구역 변환 -> 71, 121
location_x = '71'  #격자구역 x
location_y = '121' #격자구역 y

per = [api_key2, today, today_time, location_x, location_y]
k = predict_data_load_L(per)
#print(k)
#print(type(k))
#print(len(k))

0

[]
[['24' '-2.1' '1.4' '122' '2.6' '3' '0' '20' '0' '강수없음' '85' '적설없음']
 ['24' '-2.1' '1.5' '124' '2.7' '4' '0' '30' '0' '강수없음' '85' '적설없음']
 ['24' '-2' '1.7' '129' '2.7' '4' '0' '30' '0' '강수없음' '90' '적설없음']
 ['24' '-2.1' '1.8' '129' '2.8' '4' '0' '30' '0' '강수없음' '90' '적설없음']
 ['24' '-1.9' '2.2' '138' '3' '4' '0' '30' '0' '강수없음' '85' '적설없음']]


In [42]:
r = list(range(0, 11))
print(r)
print(r[:-2])
print(r[2:])

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
[0, 1, 2, 3, 4, 5, 6, 7, 8]
[2, 3, 4, 5, 6, 7, 8, 9, 10]
